In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!git clone https://github.com/huyhoang17/KIE_invoice_minimal.git

Cloning into 'KIE_invoice_minimal'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 114 (delta 0), reused 1 (delta 0), pack-reused 111
Receiving objects: 100% (114/114), 11.33 MiB | 46.23 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
!pip install -r /content/KIE_invoice_minimal/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 727 kB 30.1 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 96 kB 6.7 MB/s 
     |████████████████████████████████| 53 kB 3.0 MB/s 
     |████████████████████████████████| 70 kB 9.3 MB/s 
     |████████████████████████████████| 148 kB 75.6 MB/s 
     |████████████████████████████████| 157 kB 69.4 MB/s 
     |████████████████████████████████| 111 kB 53.8 MB/s 
     |████████████████████████████████| 149 kB 55.6 MB/s 
     |████████████████████████████████| 1.9 MB 60.3 MB/s 
ERROR: Could not find a version that satisfies the requirement dgl-cu111==0.7.1 (from versions: 0.6.0.post1, 0.6.1)
ERROR: No matching distribution found for dgl-cu111==0.7.1


In [ ]:
%cd KIE_invoice_minimal

/content/KIE_invoice_minimal


In [ ]:
import os
import time
import json
import uuid
import base64
from io import BytesIO

import imageio
import numpy as np
from PIL import Image
from fastapi import FastAPI, File, UploadFile

from backend.models import load_text_detect, load_text_recognize, load_saliency
from backend.backend_utils import (
    NpEncoder,
    run_ocr,
    make_warp_img,
    resize_and_pad,
    get_group_text_line,
)
from backend.text_detect.config import craft_config
from backend.saliency.infer import run_saliency
import configs as cf

In [ ]:
!pip install vietocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

In [ ]:
!pip install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown

In [ ]:
net = load_saliency()
text_detector = load_text_detect()
config = Cfg.load_config_from_name('vgg_transformer')
config['weights'] = 'https://drive.google.com/uc?id=13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA'
config['cnn']['pretrained']=False
config['device'] = 'cuda:0'
config['predictor']['beamsearch']=False
detector = Predictor(config)


Cached Downloading: /root/.cache/gdown/https-COLON--SLASH--SLASH-drive.google.com-SLASH-uc-QUESTION-id-EQUAL-13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA
Downloading...
From: https://drive.google.com/uc?id=13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA
To: /root/.cache/gdown/tmp_dyeb8br/dl
100%|██████████| 152M/152M [00:01<00:00, 93.4MB/s]


In [ ]:
from backend.kie.kie_utils import (
    load_gate_gcn_net,
    run_predict,
    vis_kie_pred,
    postprocess_scores,
    postprocess_write_info,
)

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
gcn_net = load_gate_gcn_net(cf.device, cf.kie_weight_path)

In [ ]:
import PIL

In [ ]:
from backend.backend_utils import create_merge_cells, get_request_api

In [ ]:
img_fp = '/content/mcocr_public_145013jwjnj.jpg'

In [ ]:
img = PIL.Image.open(img_fp,"r")

In [ ]:
import os
import time
import json
import uuid
import base64
from io import BytesIO

import imageio
import numpy as np
from PIL import Image
from fastapi import FastAPI, File, UploadFile

from backend.models import load_text_detect, load_text_recognize, load_saliency
from backend.backend_utils import (
    NpEncoder,
    run_ocr,
    make_warp_img,
    resize_and_pad,
    get_group_text_line,
)
from backend.text_detect.config import craft_config
from backend.saliency.infer import run_saliency
import configs as cf

In [ ]:
def infer(img, random_id):

    img = resize_and_pad(img, size=1024, pad=False)
    imageio.imwrite(os.path.join(cf.raw_img_dir, "{}.jpg".format(random_id)), img)

    # SALIENCY PREDICTION
    mask_img = run_saliency(net, img)
    img[~mask_img.astype(bool)] = 0.0

    # TRANSFORM AND WRAP IMAGE
    warped_img = make_warp_img(img, mask_img)
    sub_img_fp = os.path.join(cf.cropped_img_dir, "{}.jpg".format(random_id))
    imageio.imwrite(sub_img_fp, warped_img)

    # OCR
    cells, heatmap, textboxes = run_ocr(
        text_detector, detector, warped_img, craft_config
    )
    _, lines = get_group_text_line(heatmap, textboxes)
    for line_id, cell in zip(lines, cells):
        cell["group_id"] = line_id

    # CROP IMG AND UPDATE FINAL CELLS INFO
    sub_h_img, sub_w_img, _ = warped_img.shape
    img_info = dict()
    img_info["h_origin"] = sub_h_img
    img_info["w_origin"] = sub_w_img
    img_info["cells"] = cells

    with open(sub_img_fp, "rb") as image_file:
        encoded_image_string = base64.b64encode(image_file.read()).decode("utf-8")
    img_info["image"] = encoded_image_string
    return img_info


In [ ]:
total_start_time = time.time()
random_id = str(uuid.uuid4())
image = np.array(img)

In [ ]:
start = time.time()
img_info = infer(image, random_id)
delta_time = time.time() - start

- Function run_saliency run in 0.12926197's
- Function make_warp_img run in 0.00618243's
- Function run_ocr run in 1.19867826's
- Function get_group_text_line run in 0.04185939's


In [ ]:
img_info["api_runtime"] = delta_time
img_info["random_id"] = random_id
response = json.dumps(img_info, cls=NpEncoder, ensure_ascii=False)

In [ ]:
response

'{"h_origin": 992, "w_origin": 510, "cells": [{"poly": [342, 102, 374, 102, 374, 126, 342, 126], "vietocr_text": "Trần", "group_id": 0}, {"poly": [374, 102, 404, 102, 404, 126, 374, 126], "vietocr_text": "Phú", "group_id": 0}, {"poly": [282, 104, 316, 104, 316, 126, 282, 126], "vietocr_text": "anh", "group_id": 0}, {"poly": [316, 104, 342, 104, 342, 126, 316, 126], "vietocr_text": "590", "group_id": 0}, {"poly": [250, 106, 284, 106, 284, 126, 250, 126], "vietocr_text": "VM%", "group_id": 0}, {"poly": [390, 128, 428, 134, 425, 157, 386, 151], "vietocr_text": "Thuy", "group_id": 1}, {"poly": [276, 132, 308, 132, 308, 154, 276, 154], "vietocr_text": "Trần", "group_id": 1}, {"poly": [308, 132, 338, 132, 338, 154, 308, 154], "vietocr_text": "Phú", "group_id": 1}, {"poly": [342, 134, 354, 134, 354, 152, 342, 152], "vietocr_text": "P", "group_id": 1}, {"poly": [356, 132, 390, 132, 390, 154, 356, 154], "vietocr_text": "Càm", "group_id": 1}, {"poly": [230, 134, 250, 134, 250, 156, 230, 156], "v

In [ ]:
json_res = json.loads(response)

In [ ]:
json_res

{'h_origin': 992,
 'w_origin': 510,
 'cells': [{'poly': [342, 102, 374, 102, 374, 126, 342, 126],
   'vietocr_text': 'Trần',
   'group_id': 0},
  {'poly': [374, 102, 404, 102, 404, 126, 374, 126],
   'vietocr_text': 'Phú',
   'group_id': 0},
  {'poly': [282, 104, 316, 104, 316, 126, 282, 126],
   'vietocr_text': 'anh',
   'group_id': 0},
  {'poly': [316, 104, 342, 104, 342, 126, 316, 126],
   'vietocr_text': '590',
   'group_id': 0},
  {'poly': [250, 106, 284, 106, 284, 126, 250, 126],
   'vietocr_text': 'VM%',
   'group_id': 0},
  {'poly': [390, 128, 428, 134, 425, 157, 386, 151],
   'vietocr_text': 'Thuy',
   'group_id': 1},
  {'poly': [276, 132, 308, 132, 308, 154, 276, 154],
   'vietocr_text': 'Trần',
   'group_id': 1},
  {'poly': [308, 132, 338, 132, 338, 154, 308, 154],
   'vietocr_text': 'Phú',
   'group_id': 1},
  {'poly': [342, 134, 354, 134, 354, 152, 342, 152],
   'vietocr_text': 'P',
   'group_id': 1},
  {'poly': [356, 132, 390, 132, 390, 154, 356, 154],
   'vietocr_text': 

In [ ]:
api_runtime = json_res["api_runtime"]
api_random_id = json_res["random_id"]

In [ ]:
start = time.time()
imgdata = base64.b64decode(json_res["image"])


In [ ]:
imgdata

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x03\xe0\x01\xfe\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\

In [ ]:
import io

In [ ]:
pil_img = Image.open(io.BytesIO(imgdata))
img = np.array(pil_img)

In [ ]:
img

array([[[ 1,  0,  0],
        [ 1,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 1,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  1,  2],
        [10, 14, 15],
        [ 0,  2,  3]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  1,  2],
        [ 6, 10, 11],
        [ 0,  1,  2]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  4,  5],
        [ 0,  1,  2],
        [ 0,  2,  2]]], dtype=uint8)

In [ ]:
cells = json_res["cells"]

In [ ]:
cells

[{'poly': [342, 102, 374, 102, 374, 126, 342, 126],
  'vietocr_text': 'Trần',
  'group_id': 0},
 {'poly': [374, 102, 404, 102, 404, 126, 374, 126],
  'vietocr_text': 'Phú',
  'group_id': 0},
 {'poly': [282, 104, 316, 104, 316, 126, 282, 126],
  'vietocr_text': 'anh',
  'group_id': 0},
 {'poly': [316, 104, 342, 104, 342, 126, 316, 126],
  'vietocr_text': '590',
  'group_id': 0},
 {'poly': [250, 106, 284, 106, 284, 126, 250, 126],
  'vietocr_text': 'VM%',
  'group_id': 0},
 {'poly': [390, 128, 428, 134, 425, 157, 386, 151],
  'vietocr_text': 'Thuy',
  'group_id': 1},
 {'poly': [276, 132, 308, 132, 308, 154, 276, 154],
  'vietocr_text': 'Trần',
  'group_id': 1},
 {'poly': [308, 132, 338, 132, 338, 154, 308, 154],
  'vietocr_text': 'Phú',
  'group_id': 1},
 {'poly': [342, 134, 354, 134, 354, 152, 342, 152],
  'vietocr_text': 'P',
  'group_id': 1},
 {'poly': [356, 132, 390, 132, 390, 154, 356, 154],
  'vietocr_text': 'Càm',
  'group_id': 1},
 {'poly': [230, 134, 250, 134, 250, 156, 230, 156

In [ ]:
group_ids = np.array([i["group_id"] for i in json_res["cells"]])
    # merge adjacent text-boxes
merged_cells = create_merge_cells(
        detector, img, cells, group_ids, merge_text=cf.merge_text
    )
batch_scores, boxes = run_predict(gcn_net, merged_cells, device=cf.device)

- Function create_merge_cells run in 0.00436735's
- Function prepare_graph run in 0.0329628's
- Function run_predict run in 0.08746767's


In [ ]:
values, preds = postprocess_scores(
        batch_scores, score_ths=cf.score_ths, get_max=cf.get_max
    )
kie_info = postprocess_write_info(merged_cells, preds)
delta_time = time.time() - start

- Function postprocess_scores run in 0.0009439's
- Function postprocess_write_info run in 7.677e-05's


In [ ]:
save_path = os.path.join('/content/sample_data', "{}.jpg".format(api_random_id))
vis_img = vis_kie_pred(img, preds, values, boxes, save_path)

- Function vis_kie_pred run in 0.01826048's


In [ ]:
boxes

array([[250, 106, 404, 102, 404, 126, 250, 126, 154,  24],
       [230, 134, 428, 134, 425, 157, 230, 156, 198,  23],
       [ 44, 156, 182, 156, 182, 194,  44, 194, 138,  38],
       [244, 162, 412, 160, 412, 180, 244, 184, 168,  24],
       [260, 190, 394, 188, 394, 210, 260, 210, 134,  22],
       [176, 240, 336, 240, 336, 260, 176, 262, 160,  22],
       [361, 263, 471, 259, 472, 280, 362, 284, 111,  25],
       [ 42, 268, 302, 266, 302, 286,  42, 288, 260,  22],
       [342, 290, 481, 285, 482, 306, 342, 310, 140,  25],
       [ 42, 292, 126, 294, 126, 314,  42, 314,  84,  22],
       [406, 340, 473, 334, 475, 357, 406, 360,  69,  26],
       [332, 344, 354, 344, 354, 362, 332, 362,  22,  18],
       [228, 344, 292, 344, 292, 366, 228, 364,  64,  22],
       [ 42, 346, 130, 344, 130, 366,  42, 366,  88,  22],
       [ 42, 372, 430, 366, 430, 386,  42, 390, 388,  24],
       [ 42, 396, 252, 396, 252, 414,  42, 416, 210,  20],
       [416, 418, 484, 418, 484, 436, 416, 436,  68,  18